In [1]:
from gensim.models import FastText,Word2Vec
#import comman_function_analysis as cf
#import commonFunctions as cff
import os
os.environ["MKL_NUM_THREADS"] = "2" 
os.environ["NUMEXPR_NUM_THREADS"] = "2" 
os.environ["OMP_NUM_THREADS"] = "2" 
import pickle
import numpy as np
import gensim

from collections import OrderedDict

import pickle
#with open('concept_withName_dict.bin','rb') as fin:
    #concept_dict = pickle.load(fin)
    
def getAllfiles(path, endStr=''):
    """
    :param path: URL of the folder
    :startStr: starting text if specific named files are needed
    :return: a list of all files inside a folder

    """
    if endStr!= '':

        return [ y   for x in (os.walk(path)) for y in x[2] if y.endswith(endStr)  ]
    else:
        return [y for x in (os.walk(path)) for y in x[2] ]
def cosine_similarity(vec1,vec2):
    
    """
    returns the normalized cosine similarity between two vector, similar to gensim consine similarity
    """
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))


TIME_FRAME = ['1809_1970','1971_1975','1976_1980','1981_1985','1986_1990','1991_1995','1996_2000','2001_2005',\
              '2006_2010','2011_2012','2013_2015']

path = '/mnt/raid0/datasets/gvashisth/thesis/SGNS_trained_model/inc_5_yr/'
sgns_modelList = getAllfiles(path,'.bin')


sgns_model_dict = { file : 
              FastText.load_fasttext_format(path+file) 
              for file in sgns_modelList}
sgns_model_dict


{'1809_1970_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efdc3ea4390>,
 '1971_1975_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd8d834e10>,
 '1976_1980_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd8adf3828>,
 '1981_1985_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd8703ebe0>,
 '1986_1990_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd7844aeb8>,
 '1991_1995_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd75a8c5f8>,
 '1996_2000_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd5dfa0c50>,
 '2001_2005_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd55022978>,
 '2006_2010_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd3e62cf60>,
 '2011_2012_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd1309af60>,
 '2013_2015_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7efd0315b6d8>}

In [4]:
main_path ="/mnt/raid0/datasets/gvashisth/thesis/PPMI_model_uniq/"
suffix = "_explicit_ppmi.bin"

ppmi_modelList = getAllfiles(main_path,suffix)

ppm_model_dict = {}
for file in ppmi_modelList:
    with open(main_path+file, "rb") as fin :
        ppm_model_dict[file] = pickle.load(fin) 


In [246]:
%pwd

'/mnt/hdd/experiments/gvashisth/thesis/scripts'

In [247]:
import pandas as pd

treat = pd.read_csv('5_yr_10/may_treat',names=['c1','c2'])
treat.drop_duplicates(subset=['c1','c2'],inplace=True)
treat.head()

,c1,c2
0,C0000737,C0291771
1,C0000737,C0291772
2,C0000737,C0872954
3,C0000737,C1329180
4,C0000771,C0003830


In [248]:
treat.shape

(44884, 2)

In [14]:
dd= treat.iloc[:5,:]

In [ ]:
def get_true_false(dataframe, num_of_tuple):
    true_tup =  dataframe.iloc[:num_of_tuple,:]
    
    false_tup = pd.DataFrame([(x,y) for x,y in zip(np.random.choice(true_tup.c1.unique(),size=num_of_tuple*3),\
                                                      np.random.choice(true_tup.c2.unique(),size=num_of_tuple*3)) if not ((true_tup['c1'] == x) & (true_tup['c2'] == y)).any()],columns=['c1','c2'])
    return true_tup, false_tup.drop_duplicates(subset=['c1','c2']).iloc[:num_of_tuple,:]

In [279]:
from multiprocessing import Pool


In [266]:
sample[:2]

[('C0038525', 'C0977337'), ('C0038218', 'C1164736')]

In [262]:
len(sample)

134652

In [268]:
ss =  treat.iloc[:1000,:]

In [302]:
def mutli_true_false(dataframe, num_of_tuple):
    #record_list = []
    true_tup =  dataframe.iloc[:num_of_tuple,:]
    sample = [(true_tup,(x,y)) for x,y in zip(np.random.choice(true_tup.c1.unique(),size=num_of_tuple*3),\
                                                      np.random.choice(true_tup.c2.unique(),size=num_of_tuple*3))]
    record_list = pool.map(check_value,sample)
    pool.close()
    pool.join()
    #print(record_list)
    false_tup = pd.DataFrame(record_list,columns=['c1','c2'])
    return true_tup, false_tup.drop_duplicates(subset=['c1','c2']).iloc[:num_of_tuple,:]

In [299]:
def check_value(tup_rec):
    tab,tuple_row = tup_rec[0],tup_rec[1]
    if ((tab['c1'] != tuple_row[0]) & (tab['c2'] != tuple_row[1])).any() :
        return tuple_row

In [311]:
pool = Pool(20)
tr,fl = mutli_true_false(treat,num_of_tuple=44884) #get_true_false(treat,num_of_tuple=44884)

In [306]:
pd.merge(tr,fl,on=['c1','c2']).shape

(204, 2)

In [314]:
set_fl = set([tuple(line) for line in fl.values])
set_tr = set([tuple(line) for line in tr.values])
#ds1.difference(ds2)
new_fl = pd.DataFrame(list(set_fl.difference(set_tr)),columns=['c1','c2'] )
pd.merge(tr,new_fl,on=['c1','c2']).shape

(0, 2)

In [315]:
print(fl.shape)
print(tr.shape)
print(new_fl.shape)

(44884, 2)
(44884, 2)
(44685, 2)


In [156]:
def get_sgns_time(time, dataframe):
    k_sgns=time+'_fastext_10_300_5.bin' 
    model=sgns_model_dict[k_sgns]
    score_list = []
    for row in dataframe.itertuples():
        if row[1] in model.wv.vocab.keys() and row[2] in  model.wv.vocab.keys() :
            score_list.append(model.wv.similarity(row[1],row[2]) )
        else:
            score_list.append(-np.inf)
            
    dataframe['sgns_{}'.format(time)] = score_list

In [164]:
def get_ppmi_time(time, dataframe):
    k_sgns=time+'_fastext_10_300_5.bin' 
    model=sgns_model_dict[k_sgns]
    ppm_key = time+suffix
    score_list = []
    for row in dataframe.itertuples():
        if row[1] in model.wv.vocab.keys() and row[2] in  model.wv.vocab.keys() :
            score_list.append(ppm_model_dict[ppm_key].represent(row[1]).dot(ppm_model_dict[ppm_key].represent(row[2]).T)[0, 0] )
        else:
            score_list.append(-np.inf)
            
    dataframe['pmi_{}'.format(time)] = score_list

In [316]:
for time in TIME_FRAME:
    get_sgns_time(time,tr)
    get_sgns_time(time,new_fl)
    get_ppmi_time(time,tr)
    get_ppmi_time(time,new_fl)

/home/gvashisth/miniconda2/envs/new_thesis/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/gvashisth/miniconda2/envs/new_thesis/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [209]:
tr.columns

Index(['c1', 'c2', 'sgns_1809_1970', 'pmi_1809_1970', 'sgns_1971_1975',
       'pmi_1971_1975', 'sgns_1976_1980', 'pmi_1976_1980', 'sgns_1981_1985',
       'pmi_1981_1985', 'sgns_1986_1990', 'pmi_1986_1990', 'sgns_1991_1995',
       'pmi_1991_1995', 'sgns_1996_2000', 'pmi_1996_2000', 'sgns_2001_2005',
       'pmi_2001_2005', 'sgns_2006_2010', 'pmi_2006_2010', 'sgns_2011_2012',
       'pmi_2011_2012', 'sgns_2013_2015', 'pmi_2013_2015'],
      dtype='object')

In [210]:
print(tr[tr['sgns_2013_2015']!=-np.inf][:1000]['sgns_2013_2015'].mean())
print(tr[tr['sgns_2013_2015']!=-np.inf][:1000]['sgns_2013_2015'].min())
print(tr[tr['sgns_2013_2015']!=-np.inf][:1000]['sgns_2013_2015'].max())
print(tr[(tr['sgns_2013_2015']!=-np.inf) & (tr['sgns_2013_2015']<=.09)][:1000]['sgns_2013_2015'].count())

0.260028071655
-0.0219989083707
0.627778351307
32


In [203]:
print(fl[fl['sgns_2013_2015']!=-np.inf][:1000]['sgns_2013_2015'].mean())
print(fl[fl['sgns_2013_2015']!=-np.inf][:1000]['sgns_2013_2015'].min())
print(fl[fl['sgns_2013_2015']!=-np.inf][:1000]['sgns_2013_2015'].max())
print(fl[(fl['sgns_2013_2015']!=-np.inf) & (fl['sgns_2013_2015']<=.09)][:1000]['sgns_2013_2015'].count())

0.152756003184
-0.0198071133345
0.400890290737
73


In [204]:
print(tr[tr['sgns_1809_1970']!=-np.inf][:1000]['sgns_1809_1970'].mean())
print(tr[tr['sgns_1809_1970']!=-np.inf][:1000]['sgns_1809_1970'].min())
print(tr[tr['sgns_1809_1970']!=-np.inf][:1000]['sgns_1809_1970'].max())
print(tr[(tr['sgns_1809_1970']!=-np.inf) & (tr['sgns_1809_1970']<=0)][:1000]['sgns_1809_1970'].count())

0.321411861049
0.0141972340643
0.67346405983
0


In [168]:
print(tr[tr['pmi_2013_2015']!=-np.inf][:1000]['pmi_2013_2015'].mean())
print(tr[tr['pmi_2013_2015']!=-np.inf][:1000]['pmi_2013_2015'].min())
print(tr[tr['pmi_2013_2015']!=-np.inf][:1000]['pmi_2013_2015'].max())
print(tr[(tr['pmi_2013_2015']!=-np.inf) & (tr['pmi_2013_2015']<=.00)][:1000]['pmi_2013_2015'].count())

0.0129182484416
0.0
0.24246257612
747


In [221]:
tr.shape

(10000, 24)

In [222]:
#1971_1975
display(tr[(tr['sgns_1971_1975']!=-np.inf) & (tr['sgns_1971_1975']==0)].shape)


(0, 24)

In [195]:
fl[(fl['c1']=='C0005686') & (fl['c2']=='C0001714')]

,c1,c2,score_1809_1970,score_1971_1975,score_1976_1980,score_1981_1985,score_1986_1990,score_1991_1995,score_1996_2000,score_2001_2005,...,pmi_1971_1975,pmi_1976_1980,pmi_1981_1985,pmi_1986_1990,pmi_1991_1995,pmi_1996_2000,pmi_2001_2005,pmi_2006_2010,pmi_2011_2012,pmi_2013_2015


In [336]:
print("SGNS")
print("TIME, # of zeros")
#_t_sgns = ''
for i,time in enumerate(TIME_FRAME):
    #if i ==0:
    _t_sgns = tr[(tr['sgns_'+time]!=-np.inf)].sample(n=3000) #['sgns_'+time].count() & (tr['sgns_'+time]==.00)
    #print(_t_sgns.shape)#_t_sgns[(_t_sgns['pmi_'+time]!=-np.inf) & (_t_sgns['pmi_'+time]<=.00)]['pmi_'+time].count()
        #print('hh')
    print("scount,{},{} ".format(time,_t_sgns[_t_sgns['sgns_'+time]<=.05]['sgns_'+time].count()))
    print("pcount,{},{}".format(time,_t_sgns[_t_sgns['pmi_'+time]==0]['pmi_'+time].count()))
    print("smean,{},{},{}".format(time,_t_sgns['sgns_'+time].mean(),_t_sgns['sgns_'+time].max()))
    print("pmean,{},{},{} ".format(time,_t_sgns['pmi_'+time].mean(),_t_sgns['pmi_'+time].max()))

SGNS
TIME, # of zeros
hh
scount,1809_1970,5 
pcount,1809_1970,221
smean,1809_1970,0.3311867487039417,0.7543472051620483
pmean,1809_1970,0.025915743040875044,0.2857647771955965 
scount,1971_1975,3 
pcount,1971_1975,662
smean,1971_1975,0.3082364931019644,0.705193817615509
pmean,1971_1975,0.020910345899059168,0.35414775148968136 
scount,1976_1980,12 
pcount,1976_1980,356
smean,1976_1980,0.29186973847448827,0.6974166631698608
pmean,1976_1980,0.024491521027427295,0.376214549943841 
scount,1981_1985,16 
pcount,1981_1985,357
smean,1981_1985,0.28621718647393085,0.670173168182373
pmean,1981_1985,0.025364236782319412,0.3296228201313084 
scount,1986_1990,14 
pcount,1986_1990,313
smean,1986_1990,0.2791112131004532,0.7014037370681763
pmean,1986_1990,0.024527483685168,0.2923105592844246 
scount,1991_1995,17 
pcount,1991_1995,425
smean,1991_1995,0.2728804139830172,0.672159731388092
pmean,1991_1995,0.022304666335834833,0.3245429825992315 
scount,1996_2000,12 
pcount,1996_2000,415
smean,1996_2000,0.270

In [331]:
# false
print("SGNS")
print("TIME, # of zeros")
for time in TIME_FRAME:
    _f_sgns = new_fl[(new_fl['sgns_'+time]!=-np.inf)].sample(n=3000)
    print("scount,{},{} ".format(time,_f_sgns[_f_sgns['sgns_'+time]<=.05]['sgns_'+time].count()))
#print("PPMI")
#print("TIME, # of zeros")
#for time in TIME_FRAME: _f_sgns[(_f_sgns['pmi_'+time]!=-np.inf) & (_f_sgns['pmi_'+time]==.00)]['pmi_'+time].count()    
    print("pcount,{},{}".format(time,_f_sgns[_f_sgns['pmi_'+time]==0]['pmi_'+time].count()))
    print("smean,{},{},{}".format(time,_f_sgns['sgns_'+time].mean(),_f_sgns['sgns_'+time].max()))
    print("pmean,{},{},{} ".format(time,_f_sgns['pmi_'+time].mean(),_f_sgns['pmi_'+time].max()))
    
    


SGNS
TIME, # of zeros
scount,1809_1970,56 
pcount,1809_1970,573
smean,1809_1970,0.20791763930705687,0.5881251096725464
pmean,1809_1970,0.003485299286974925,0.17094912177867827 
scount,1971_1975,52 
pcount,1971_1975,984
smean,1971_1975,0.19742975264042617,0.5964272022247314
pmean,1971_1975,0.003107798210948336,0.08599810844941813 
scount,1976_1980,106 
pcount,1976_1980,680
smean,1976_1980,0.16828050282752763,0.5639092922210693
pmean,1976_1980,0.0027488312398833634,0.1344159810396496 
scount,1981_1985,137 
pcount,1981_1985,663
smean,1981_1985,0.15723529312697548,0.5304141640663147
pmean,1981_1985,0.002579327870445759,0.1499260954958605 
scount,1986_1990,135 
pcount,1986_1990,729
smean,1986_1990,0.15478395048358168,0.5534582138061523
pmean,1986_1990,0.002387576435368064,0.13862172068296955 
scount,1991_1995,150 
pcount,1991_1995,761
smean,1991_1995,0.14907450615552564,0.5052852630615234
pmean,1991_1995,0.00216061134271263,0.07294507012679965 
scount,1996_2000,157 
pcount,1996_2000,780
sme

In [205]:
tr.columns

Index(['c1', 'c2', 'score_1809_1970', 'score_1971_1975', 'score_1976_1980',
       'score_1981_1985', 'score_1986_1990', 'score_1991_1995',
       'score_1996_2000', 'score_2001_2005', 'score_2006_2010',
       'score_2011_2012', 'score_2013_2015', 'sgns_1809_1970',
       'sgns_1971_1975', 'sgns_1976_1980', 'sgns_1981_1985', 'sgns_1986_1990',
       'sgns_1991_1995', 'sgns_1996_2000', 'sgns_2001_2005', 'sgns_2006_2010',
       'sgns_2011_2012', 'sgns_2013_2015', 'pmi_1809_1970', 'pmi_1971_1975',
       'pmi_1976_1980', 'pmi_1981_1985', 'pmi_1986_1990', 'pmi_1991_1995',
       'pmi_1996_2000', 'pmi_2001_2005', 'pmi_2006_2010', 'pmi_2011_2012',
       'pmi_2013_2015'],
      dtype='object')

In [322]:
print("SGNS")
print("TIME, mean, max")
for time in TIME_FRAME:
    _t_sgns = tr[(tr['sgns_'+time]!=-np.inf)].sample(n=1000)
    print("s,{},{},{}".format(time,_t_sgns['sgns_'+time].mean(),_t_sgns['sgns_'+time].max()))
#print("PPMI")
#print("TIME, mean, std")
#for time in TIME_FRAME:     
    print("p,{},{},{} ".format(time,_t_sgns['pmi_'+time].mean(),_t_sgns['pmi_'+time].max()))

SGNS
TIME, mean, max
s,1809_1970,0.3300205420255661,0.6775633096694946
p,1809_1970,0.025492352910969115,0.2593108902398268 
s,1971_1975,0.31563577457889913,0.72144615650177
p,1971_1975,0.020825563405566003,0.33427211433678056 
s,1976_1980,0.3087928900979459,0.6780615448951721
p,1976_1980,0.026527824821518584,0.26859233670840593 
s,1981_1985,0.30712686674669387,0.670173168182373
p,1981_1985,0.02907402191812182,0.2812335263049411 
s,1986_1990,0.30453384345024825,0.6468294262886047
p,1986_1990,0.029876612640828033,0.3104281458389142 
s,1991_1995,0.3002914763391018,0.656867265701294
p,1991_1995,0.025700713029063956,0.34830384692691874 
s,1996_2000,0.30331080087274315,0.7085130214691162
p,1996_2000,0.02703572799382744,0.27852401772580365 
s,2001_2005,0.2873845221847296,0.6524094343185425
p,2001_2005,0.024463823847898077,0.29510709085761244 
s,2006_2010,0.27240155317261816,0.6822232604026794
p,2006_2010,0.021288466824764887,0.2177301232239228 
s,2011_2012,0.27103210531547667,0.73042023181915

In [ ]:
print("SGNS")
print("TIME, mean, std")
for time in TIME_FRAME:
    _f_sgns = fl[(fl['sgns_'+time]!=-np.inf)].sample(n=1000)
    print("s,{},{},{}".format(time,_f_sgns['sgns_'+time].mean(),_f_sgns['sgns_'+time].std()))
#print("PPMI")
#print("TIME, mean, std")
#for time in TIME_FRAME:     
    print("p,{},{},{} ".format(time,_f_sgns[_f_sgns['pmi_'+time]!=-np.inf]['pmi_'+time].mean(),_f_sgns[_f_sgns['pmi_'+time]!=-np.inf]['pmi_'+time].std()))

In [186]:
fl.columns

Index(['c1', 'c2', 'score_1809_1970', 'score_1971_1975', 'score_1976_1980',
       'score_1981_1985', 'score_1986_1990', 'score_1991_1995',
       'score_1996_2000', 'score_2001_2005', 'score_2006_2010',
       'score_2011_2012', 'score_2013_2015', 'sgns_1809_1970',
       'sgns_1971_1975', 'sgns_1976_1980', 'sgns_1981_1985', 'sgns_1986_1990',
       'sgns_1991_1995', 'sgns_1996_2000', 'sgns_2001_2005', 'sgns_2006_2010',
       'sgns_2011_2012', 'sgns_2013_2015', 'pmi_1809_1970', 'pmi_1971_1975',
       'pmi_1976_1980', 'pmi_1981_1985', 'pmi_1986_1990', 'pmi_1991_1995',
       'pmi_1996_2000', 'pmi_2001_2005', 'pmi_2006_2010', 'pmi_2011_2012',
       'pmi_2013_2015'],
      dtype='object')